In [2]:
import cupyx.scipy as cuxsp
import numpy as np
import cupy as cp
import scipy as sp
from numba import njit

In [199]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
print(mempool.used_bytes())             
print(mempool.total_bytes())           
print(pinned_mempool.n_free_blocks())  


5058679296
5058790912
0


In [198]:
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()

## Preparation:
<br>
<br>
對資料長度 120*120*120 三維陣列計算discrete laplacian。<br>
ipython %%timeit做測速。<br>
<br>
<br>

In [3]:
Nx = 120
Ny = 120
Nz = 120
Lx = 1
Ly = 1
Lz = 1
x = np.linspace(-Lx,Lx,Nx)
y = np.linspace(-Ly,Ly,Ny)
z = np.linspace(-Lz,Lz,Nz)
dx = np.diff(x)[0]
dy = np.diff(y)[0]
dz = np.diff(z)[0]
[X,Y,Z] = np.meshgrid(x,y,z)

w = X**2 + Y**2 + Z**2


## Result

<table border="1">
    <caption> Execution time </caption>
    <thead>
        <tr>
            <th rowspan=2>Machine</th>
            <th colspan=3
                style="text-align:center">CPU</th>
            <th rowspan=3
                colspan=3
                style="text-align:center">GPU</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan=4>Local</td>
            <td> brutal method </td>
            <td><strong>brutal method<br>(njit compiled) </strong></td>
            <td> fortran vectorization </td>
        </tr>
        <tr>
            <td> 4.57 s ± 60.7 ms </td>
            <td><strong>6.21 ms ± 136 µs  </td></strong>
            <td> 9 ms ± 282 µs  </td>
        </tr>
        <tr>
            <td>Vetorization<br>(njit compiled)</td>
            <td>tensor product</td>
            <td>sparse matrix</td>
            <td><strong>vetorization</td></strong>
            <td>tensor product</td>
            <td>sparse matrix</td>
        </tr>
        <tr>
            <td> 24.9 ms ± 1.03 ms </td>
            <td> 337 ms ± 3.86 ms </td>
            <td> 153 ms ± 2.13 ms </td>
            <td><strong> 1.4 ms ± 2.74 µs </td></strong>
            <td> 4.58 ms ± 8.93 µs </td>
            <td> 588 ms ± 46.9 ms </td>
        </tr>
        <tr>
            <td rowspan=4>Master</td>
            <td> brutal method </td>
            <td><strong>brutal method<br>(njit compiled) </td></strong>
            <td> fortran vectorization </td>
        </tr>
        <tr>
            <td> 3.73 s ± 28.3 ms </td>
            <td> <strong>5.37 ms ± 94.9 µs  </td></strong>
            <td> 6.33 ms ± 99.3 µs </td>
        </tr>
        <tr>
            <td>vetorization<br>(njit compiled)</td>
            <td>tensor product</td>
            <td>sparse matrix</td>
            <td><strong>vetorization</td></strong>
            <td>tensor product</td>
            <td>sparse matrix</td>
        </tr>
        <tr>
            <td> 20.9 ms ± 644 µs  </td>
            <td> 260 ms ± 2.76 ms </td>
            <td> 82 ms ± 866 µs </td>
            <td><strong> 965 µs ± 179 ns </td></strong>
            <td> 3.23 ms ± 8.35 µs  </td>
            <td> 177 ms ± 1.06 ms </td>
        </tr>
    </tbody>
        
python vectorization: 31 ms ± 1.93 ms<br>
fortran brutal method : 18.1 ms ± 766 µs


brutal method

In [6]:
# @njit(fastmath=True, nogil=True)
def brutal(w, dx,dy,dz):
    lap = np.zeros_like(w)
    for i in range(1,w.shape[0]-1):
        for j in range(1, w.shape[1]-1):
            for k in range(1, w.shape[2]-1):
                lap[i,j,k] = (1/dx)**2 * ( w[i,j+1,k] - 2*w[i,j,k] + w[i,j-1,k]) + \
                            (1/dy)**2 * ( w[i+1,j,k] - 2*w[i,j,k] + w[i-1,j,k]) + \
                            (1/dz)**2 * ( w[i,j,k+1] - 2*w[i,j,k] + w[i,j,k-1])
                    
    return lap

In [7]:
%%timeit
brutal(w,dx,dy,dz)

3.73 s ± 28.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


compiled loop method

In [8]:
@njit(fastmath=True, nogil=True)
def loop(w, _dx, _dy, _dz):
    """ Calculate the del2 of the array w=[].
        Note that the boundary points aren't handled """
    lap = np.zeros_like(w)
    for z in range(1,w.shape[2]-1):
        lap[:, :, z] = (1/_dz)**2 * ( w[:, :, z+1] - 2*w[:, :, z] + w[:, :, z-1] )
    for y in range(1,w.shape[0]-1):
        lap[y, :,:] = lap[y, :, :] + (1/_dy)**2 * ( w[y+1, :, :] - 2*w[y, :, :] + w[y-1, :, :] )
    for x in range(1,w.shape[1]-1):
        lap[:, x,:] = lap[:, x, :] + (1/_dx)**2 * ( w[:, x+1, :] - 2*w[:, x, :] + w[:, x-1, :] )
    return lap


In [9]:
%%timeit
loop(w,dx,dy,dz)

20.9 ms ± 644 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


>compiled vectorized method

In [10]:
@njit(fastmath=True, nogil=True)
def vector_CPU(w, dx, dy, dz):
    u=np.zeros_like(w)
    u[1:w.shape[0]-1,1:w.shape[1]-1,1:w.shape[2]-1] = (
        (1/dy**2)*(
                w[2:w.shape[0],   1:w.shape[1]-1, 1:w.shape[2]-1] 
            - 2*w[1:w.shape[0]-1, 1:w.shape[1]-1, 1:w.shape[2]-1] 
            + w[0:w.shape[0]-2, 1:w.shape[1]-1, 1:w.shape[2]-1])
        +(1/dx**2)*(
                w[1:w.shape[0]-1, 2:w.shape[1],   1:w.shape[2]-1] 
            - 2*w[1:w.shape[0]-1, 1:w.shape[1]-1, 1:w.shape[2]-1] 
            + w[1:w.shape[0]-1, 0:w.shape[1]-2, 1:w.shape[2]-1])
        +(1/dz**2)*(
                w[1:w.shape[0]-1, 1:w.shape[1]-1, 2:w.shape[2]]
            - 2*w[1:w.shape[0]-1, 1:w.shape[1]-1, 1:w.shape[2]-1] 
            + w[1:w.shape[0]-1, 1:w.shape[1]-1, 0:w.shape[2]-2]))
    return u

In [11]:
%%timeit
vector_CPU(w,dx,dy,dz)

21.4 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


density matrix tensor method on CPU


In [12]:
N, _, _ = w.shape 
D = np.eye(N,k=-1) + -2*np.eye(N,k=0) + np.eye(N,k=1)
D[0,:] = D[-1,:] = np.zeros((1,N))
lap = np.zeros_like(w)

In [13]:
%%timeit
lap = (1/dy**2) * np.einsum('ij,jkl->ikl', D, w) + \
        (1/dx**2) * np.einsum('ij,kjl->kil', D, w) + \
        (1/dz**2) * np.einsum('ij,klj->kli', D, w)

260 ms ± 2.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


sparse matrix on CPU

In [16]:
N, _, _ = w.shape
ex = np.ones(N)
values = np.repeat(ex,3).reshape(3,N)
values[1,:] *= -2
values[1,0] = values[1,-1]  = 0  
values[0,-2] = values[2,1] =  0
D = sp.sparse.dia_matrix((values,[-1,0,1]),shape=(N,N))
lap = np.zeros_like(w)
# D = D.A
# i=0

In [17]:
%%timeit
for i in range(1,N-1):
    lap[:, :, i] += (1/dy)**2* (D @ w[:,:,i]) + (1/dx)**2 * (D @ w[:,:,i].T).T
    lap[i, :, :] += (1/dz)**2* (D @ w[i,:,:].T).T

82 ms ± 866 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


vectorized method on GPU

In [27]:
ww = cp.array(w)
Ny,Nx,Nz=ww.shape
# boundary is zero
u=cp.zeros_like(ww)

In [22]:
type(u)

cupy._core.core.ndarray

In [28]:
%%timeit
u[1:Ny-1,1:Nx-1,1:Nz-1] = (
    (1/dy**2)*(
            ww[2:Ny,   1:Nx-1, 1:Nz-1] 
        - 2*ww[1:Ny-1, 1:Nx-1, 1:Nz-1] 
        + ww[0:Ny-2, 1:Nx-1, 1:Nz-1])
    +(1/dx**2)*(
            ww[1:Ny-1, 2:Nx,   1:Nz-1] 
        - 2*ww[1:Ny-1, 1:Nx-1, 1:Nz-1] 
        + ww[1:Ny-1, 0:Nx-2, 1:Nz-1])
    +(1/dz**2)*(
            ww[1:Ny-1, 1:Nx-1, 2:Nz]
        - 2*ww[1:Ny-1, 1:Nx-1, 1:Nz-1] 
        + ww[1:Ny-1, 1:Nx-1, 0:Nz-2]))

965 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


dense matrix tensor method on GPU

In [33]:
ww = cp.array(w)
N, _, _ = ww.shape 
D = cp.eye(N,k=-1) + -2*cp.eye(N,k=0) + cp.eye(N,k=1)
D[0,:] = D[-1,:] = cp.zeros((1,Nx))
lap = cp.zeros_like(ww)

In [34]:
%%timeit
lap = (1/dy**2) * cp.einsum('ij,jkl->ikl', D, ww) + \
          (1/dx**2) * cp.einsum('ij,kjl->kil', D, ww) + \
          (1/dz**2) * cp.einsum('ij,klj->kli', D, ww)

3.23 ms ± 8.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


sparse matrix on GPU

In [37]:
ww = cp.array(w)
N,_,_ = ww.shape
ex = cp.ones(N)
values = cp.repeat(ex,3).reshape(3,N)
values[1,:] *= -2
values[1,0] = values[1,-1]  = 0  
values[0,-2] = values[2,0] =  0
D = cuxsp.sparse.diags(values,[-1,0,1],shape=(N,N))
lap = cp.zeros_like(ww)
# i = 0
# D = D.A

In [38]:
%%timeit
for i in range(1,N-1): 
    lap[:, :, i] += (1/dy)**2* (D @ ww[:,:,i]) + (1/dx)**2 * ( D @ ww[:,:,i].T ).T
    lap[i, :, :] += (1/dz)**2* (D @ ww[i,:,:].T).T

177 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
import fdel2

In [40]:
pwd

'/home/quojinhao/QOPTICS/Simulation/Practice'

In [43]:
%%timeit
fdel2.del2_loop(w,dx,dy,dz)

18.1 ms ± 69.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%%timeit
fdel2.del2_ele(w,dx,dy,dz)

6.33 ms ± 99.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
